In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time

import os
import numpy as np
# import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, GRU,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
import pandas as pd

In [77]:
# get texts data
category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)
# train_df_sample = pd.concat([train_pickle_df.text]).sample(frac=1)

In [78]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180425.model")
# print(type(answer))
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
# print(wvv_keys_list[:10]) #['櫻花林', '好比', '考科', '床上', '一點現', '記住', '寶寶的', '柔嫩', '不規則', '朴智妍']

In [79]:
train_texts = train_df_sample.values
label_list = to_categorical(train_df_sample.category)

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values

In [80]:
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])

# train_texts_index = train_texts_list

# print(len(train_texts_list), train_texts_list[0])

In [81]:
texts_list = []
for text in train_texts_list:
    texts_list.append(text)
    
for text in test_texts:
    texts_list.append(text)

In [82]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_list)
sequences = tokenizer.texts_to_sequences(texts_list)
# max_doc_word_length = max(len(l) for l in train_texts)
max_doc_word_length = 200
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='pre')
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))


del texts_list,train_texts_list,test_texts

Found 218242 unique tokens


In [83]:
word_counts = tokenizer.word_counts
print(type(word_counts), len(word_counts))
new_word_counts = list(word_counts.items())
print(type(new_word_counts), len(new_word_counts))
new_word_counts = sorted(new_word_counts, key=lambda s: s[1], reverse=True)
ii = 0
print(len(word_index))
# new_word_counts[20:41142]
for word in new_word_counts:
    if word[1] < 4:
        del word_index[word[0]]
    if word[1] > 4000:
#         print(word[0], word[1])
        del word_index[word[0]]
print(len(wvv_keys_list))
print(len(word_index))

<class 'collections.OrderedDict'> 218242
<class 'list'> 218242
218242
34453
41796


In [84]:
# others_words = []
# for k,w in word_index.items():
#     if w >41797:
# #         del word_index[k]
#         others_words.append(k)
#         print(k,w)
# for w in others_words:
#     del word_index[w]
# print("-------------") 

word_index_list= []
for key, value in word_index.items():
    temp = [key,value]
    word_index_list.append(key)
del word_index
new_word_index = {word_index_list[i]: i for i in range(0, len(word_index_list))}


In [87]:
print(len(new_word_index))
print(word_index_list[:5])
print(new_word_index['戴隱形'])
# for k,w in word_index.items():
# #     if w >41797:
# # #         del word_index[k]
# #         others_words.append(k)
#         print(k,w)

41796
['張', '戴隱形', '寫實', '聊得', '北美票房']
1


In [88]:
vocab_size = len(new_word_index) + 1
# create a weight matrix for words in training docs
answer_vector_size = answer.vector_size
embedding_matrix = np.zeros((vocab_size, answer_vector_size))
embed_i = 1
for word in new_word_index:
    if word in wvv_keys_list:
        embedding_vector = answer[word]
        if embedding_vector is not None:
            embedding_matrix[embed_i] = embedding_vector
    else:
        embedding_matrix[embed_i] = np.zeros((1, answer_vector_size))
    embed_i += 1
print(len(embedding_matrix))

41797


In [89]:
del answer

In [101]:
embedding_layer = Embedding(input_dim= embedding_matrix.shape[0],
                            output_dim= embedding_matrix.shape[1],
                            weights=[embedding_matrix], 
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(GRU(16))    
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 250)         10449250  
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                12816     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               1700      
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1010      
Total params: 10,474,876
Trainable params: 25,626
Non-trainable params: 10,449,250
_________________________________________________________________


In [102]:
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(new_word_index[word])
            except:
                new_doc.append(0)
#         new_doc_arr = np.array(new_doc).reshape(1, max_doc_word_length)
        new_doc_arr = np.array(new_doc)
        new_corpus.append( new_doc_arr)
    return np.array(new_corpus)

In [103]:
X_train_texts = text_to_index(train_df_sample.text)
X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

In [104]:
# X_train_texts_ = list(X_train_texts)
print(len(X_train), type(X_train))
print(len(X_train[0]), type(X_train[0]))
print(X_train[0].shape)
print(X_train[1].shape)

9000 <class 'numpy.ndarray'>
200 <class 'numpy.ndarray'>
(200,)
(200,)


In [108]:
history = model.fit(x = X_train, y = label_list, 
                    validation_split=0.1, 
                    batch_size= 4050,
                    epochs = 100, verbose = 1)

Train on 8100 samples, validate on 900 samples
Epoch 1/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1133 - acc: 0.6101 - val_loss: 1.8956 - val_acc: 0.4156
Epoch 2/100
8100/8100 [==============================] - 11s 1ms/step - loss: 1.1221 - acc: 0.6041 - val_loss: 1.9104 - val_acc: 0.4122
Epoch 3/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1122 - acc: 0.6141 - val_loss: 1.9040 - val_acc: 0.4056
Epoch 4/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1070 - acc: 0.6121 - val_loss: 1.8971 - val_acc: 0.4167
Epoch 5/100
8100/8100 [==============================] - 11s 1ms/step - loss: 1.1075 - acc: 0.6143 - val_loss: 1.9118 - val_acc: 0.4022
Epoch 6/100
8100/8100 [==============================] - 11s 1ms/step - loss: 1.1120 - acc: 0.6093 - val_loss: 1.8952 - val_acc: 0.4056
Epoch 7/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1013 - acc: 0.6154 - val_loss: 1.8998 - val_acc: 0.4067
E

In [109]:
# loss_func_name = 'categorical_crossentropy'
# for value in history.history["categorical_crossentropy"]:
#     print(history)
model.save('my_model.h5') 

In [110]:
# # evaluate the model
# loss_accuracy = model.evaluate(sequences1[0:200], label_list[0:200], verbose=1)
# print(type(loss_accuracy), loss_accuracy)

In [111]:
# test_tokenizer = Tokenizer()
# test_tokenizer.fit_on_texts(test_texts)
# test_sequences = test_tokenizer.texts_to_sequences(test_texts)
# test_sequences1 = pad_sequences(test_sequences, maxlen=max_doc_word_length, padding='post')
# X_train_texts = text_to_index(train_df_sample.text)
# X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

Y_sequences1 = text_to_index(test_pickle_df.text)

Y_test = pad_sequences(Y_sequences1, maxlen=max_doc_word_length)
print(Y_test[732])

[    0     0     0     0     0     0     0     0     0     0     0     0
 11100 32721  3549  7078 13678 34042 37928 17456  4631 22489 19390     0
 26520     0   953 25497     0  5859 15885     0 19828 25497   559     0
 33028 13253 22746 27070 29849 30184  9925  7329 23712  5695  7747  6630
 13253 22490     0 10247     0 41566     0 33028 21683  9817  9572     0
 16997 26563 41219     0 12543 13625 27070 29849 10699 14817 23359 33284
 41219 25565 11303 27070 38683 20474 24195 38854 40905 35139 34936 29098
 18360 12032 36316 37300  5610 34936     0 34259 16561     0 35083 10247
 13070  5666 32104 41219 33028 13253 38336 41219     0     0 13353 33115
 40895 29762 21275 29762  7698 13267 33028 13253 38336     0 32104 10247
 16721  7190  4722 23570     0 11676 32551  2851  4722 24195 13150     0
  4112 19390  9373 33922 25816  5666  5934   471 10247     0 30729 37928
 17456 33096 12497 19708 18287 27070 21390 11244 22290 39026 14370 31826
 32149     0     0 12579 20226 32702     0 23351 19

In [112]:
predict_res = model.predict(Y_test[0:], verbose=1)
# print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(np.argmax(pre_res))

    
# 'Japan_Travel': 0, 'KR_ENTERTAIN' 娱乐: 1, 'Makeup' 化妆 : 2, 'Tech_Job':  3, 'WomenTalk': 4, 
# 'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}
# 7, 9, 4, 5,    1/2, 1/8, 3/7, 1, 7, 0
# 0, 9, 6, x(生活), 9,  9, 9, 7/3, 7, 1
print(final_res)

1000/1000 [==============================] - 2s 2ms/step
[0, 9, 2, 0, 8, 8, 5, 7, 3, 6, 0, 4, 2, 8, 4, 9, 9, 3, 1, 9, 3, 7, 9, 2, 1, 4, 8, 4, 8, 5, 0, 0, 0, 0, 0, 6, 0, 4, 1, 8, 4, 4, 3, 5, 8, 9, 6, 0, 6, 7, 0, 5, 7, 2, 5, 4, 5, 9, 2, 0, 6, 6, 0, 3, 7, 4, 8, 5, 0, 3, 0, 0, 7, 7, 8, 6, 7, 4, 3, 2, 2, 9, 1, 1, 5, 2, 2, 7, 4, 0, 6, 3, 4, 0, 9, 3, 6, 2, 4, 4, 7, 8, 2, 9, 7, 4, 7, 2, 2, 3, 8, 7, 5, 8, 4, 0, 9, 7, 4, 9, 8, 9, 5, 6, 3, 8, 8, 1, 6, 7, 7, 6, 6, 3, 7, 4, 9, 4, 1, 0, 4, 7, 3, 7, 2, 6, 2, 3, 0, 6, 7, 3, 9, 6, 7, 2, 6, 5, 2, 9, 6, 6, 8, 6, 2, 7, 8, 5, 2, 7, 7, 9, 8, 4, 7, 8, 2, 2, 4, 4, 1, 4, 1, 2, 2, 7, 2, 3, 0, 0, 3, 4, 8, 0, 8, 3, 8, 8, 6, 8, 2, 7, 3, 6, 3, 3, 8, 6, 6, 9, 4, 2, 2, 2, 3, 0, 2, 3, 2, 4, 0, 4, 9, 7, 9, 4, 1, 6, 4, 9, 4, 1, 0, 4, 7, 6, 0, 8, 3, 8, 9, 5, 6, 6, 2, 2, 2, 7, 8, 3, 9, 0, 0, 4, 9, 6, 4, 6, 4, 1, 1, 8, 1, 6, 4, 2, 6, 0, 7, 3, 7, 6, 2, 8, 6, 1, 7, 9, 9, 0, 7, 4, 4, 7, 8, 0, 4, 5, 2, 2, 5, 0, 0, 8, 6, 3, 5, 4, 7, 9, 6, 8, 0, 6, 0, 6, 5, 9, 3, 9, 2, 8, 2, 8, 

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1